# 7 How Classes Work 

### 7.1 Deconstructing the Class Statement


- The `class` statement starts a block of code and creates a new
  namespace.  All namespace changes in the block, e.g. assignment and
  function definitions, are made in that new namespace.  Finally it
  adds the class name to the namespace where the class statement
  appears.

- Instances of a class are created by calling the class:
  `ClassName()` or `ClassName(args)`.

- `ClassName.__init__(<new object>, ...)` is called automatically, and
  is passed the instance of the class already created by a call to the
  `__new__` method.

- Accessing an attribute `method_name` on a class instance returns
  a *method object*, if `method_name` references a method (in
  `ClassName` or its superclasses).  A method object binds the class
  instance as the first argument to the method.


In [ ]:
class Number:  # In Python 2.x use "class Number(object):"
    """A number class."""
    __version__ = '1.0'
    
    def __init__(self, amount):
        self.amount = amount
    
    def add(self, value):
        """Add a value to the number."""
        print('Call: add({!r}, {})'.format(self, value))
        return self.amount + value

In [ ]:
Number

In [ ]:
Number.__version__

In [ ]:
Number.__doc__

In [ ]:
help(Number)

In [ ]:
Number.__init__

In [ ]:
Number.add

In [ ]:
dir(Number)

In [ ]:
def dir_public(obj):
    return [n for n in dir(obj) if not n.startswith('__')]

In [ ]:
dir_public(Number)

In [ ]:
number2 = Number(2)

In [ ]:
number2.amount

In [ ]:
number2

In [ ]:
number2.__init__

In [ ]:
number2.add

In [ ]:
dir_public(number2)

In [ ]:
set(dir(number2)) ^ set(dir(Number))  # symmetric_difference

In [ ]:
number2.__dict__

In [ ]:
Number.__dict__

In [ ]:
'add' in Number.__dict__

In [ ]:
number2.add

In [ ]:
number2.add(3)

  Here's some unusual code ahead which will help us think carefully
about how Python works.

In [ ]:
Number.add

In [ ]:
def add(self, value):  # Earlier definition
        return self.amount + value

In [ ]:
Number.add(2)

In [ ]:
Number.add(2, 3)

In [ ]:
Number.add(number2, 3)

In [ ]:
number2.add(3)

In [ ]:
def __init__(self, amount):  # Earlier definition
    self.amount = amount

In [ ]:
Number.__init__

In [ ]:
help(Number.__init__)

  Here's some code that's downright risky, but instructive.  You
should never need to do this in your code.

In [ ]:
def set_double_amount(number, amount):
    number.amount = 2 * amount

In [ ]:
Number.__init__ = set_double_amount

In [ ]:
Number.__init__

In [ ]:
help(Number.__init__)

In [ ]:
number4 = Number(2)

In [ ]:
number4.amount

In [ ]:
number4.add(5)

In [ ]:
number4.__init__

In [ ]:
number2.__init__

In [ ]:
def multiply_by(number, value):
    return number.amount * value

  Let's add a `mul` method.  However, I will intentionally make a mistake.

In [ ]:
number4.mul = multiply_by

In [ ]:
number4.mul

In [ ]:
number4.mul(5)

In [ ]:
number4.mul(number4, 5)

Where's the mistake?

In [ ]:
number10 = Number(5)

In [ ]:
number10.mul

In [ ]:
dir_public(number10)

In [ ]:
dir_public(Number)

In [ ]:
dir_public(number4)

In [ ]:
Number.mul = multiply_by

In [ ]:
number10.mul(5)

In [ ]:
number4.mul(5)

In [ ]:
dir_public(number4)

In [ ]:
number4.__dict__

In [ ]:
del number4.mul

In [ ]:
number4.__dict__

In [ ]:
dir_public(number4)

In [ ]:
number4.mul

In [ ]:
Number.mul

In [ ]:
number4.mul(5)

  Let's look behind the curtain to see how class instances work in Python.

In [ ]:
Number

In [ ]:
number4

In [ ]:
Number.add

In [ ]:
number4.add

  Bound methods are handy.

In [ ]:
add_to_4 = number4.add

In [ ]:
add_to_4(6)

In [ ]:
dir_public(number4)

In [ ]:
dir(number4.add)

In [ ]:
dir_public(number4.add)

In [ ]:
set(dir(number4.add)) - set(dir(Number.add))

In [ ]:
number4.add.__self__

In [ ]:
number4.add.__self__ is number4

In [ ]:
number4.add.__func__

In [ ]:
number4.add.__func__ is Number.add

In [ ]:
number4.add.__func__ is number10.add.__func__

In [ ]:
number4.add(5)

  So here's approximately how Python executes `number4.add(5)`:

In [ ]:
number4.add.__func__(number4.add.__self__, 5)

### 7.2 Creating Classes with the type Function


"The class statement is just a way to call a function, take the
result, and put it into a namespace." -- Glyph Lefkowitz in *Turtles
All The Way Down: Demystifying Deferreds, Decorators, and
Declarations* at PyCon 2010

`type(name, bases, dict)` is the default function that gets called
when Python read a `class` statement.

In [ ]:
print(type.__doc__)

  Let's use the type function to build a class.

In [ ]:
def init(self, amount):
    self.amount = amount

In [ ]:
def add(self, value):
    """Add a value to the number."""
    print('Call: add({!r}, {})'.format(self, value))
    return self.amount + value

In [ ]:
Number = type(
    'Number', (object,),
    dict(__version__='1.0', __init__=init, add=add))

In [ ]:
number3 = Number(3)

In [ ]:
type(number3)

In [ ]:
number3.__class__

In [ ]:
number3.__dict__

In [ ]:
number3.amount

In [ ]:
number3.add(4)

  Remember, here's the normal way to create a class:

In [ ]:
class Number:
    __version__='1.0'
    
    def __init__(self, amount):
        self.amount = amount
    
    def add(self, value):
        return self.amount + value


We can customize how classes get created.  
https://docs.python.org/3/reference/datamodel.html#customizing-class-creation

> By default, classes are constructed using type(). The class body is
> executed in a new namespace and the class name is bound locally to
> the result of type(name, bases, namespace).

> The class creation process can be customised by passing the
> metaclass keyword argument in the class definition line, or by
> inheriting from an existing class that included such an argument.


  The following makes explicit that the `metaclass`, i.e. the
callable that Python should use to create a class, is the built-in
function `type`.

In [ ]:
class Number(metaclass=type):
    def __init__(self, amount):
        self.amount = amount

### 7.3 Exercises: The Class Statement

Test your understanding of the mechanics of class creation with some
very unconventional uses of those mechanics.

  What does the following code do?  Note that `return_5` ignores
arguments passed to it.

In [ ]:
def return_5(name, bases, namespace):
    print('Called return_5({!r})'.format((name, bases, namespace)))
    return 5 

In [ ]:
return_5(None, None, None)

In [ ]:
x = return_5(None, None, None)

In [ ]:
x

In [ ]:
type(x)

  The syntax for specifying a metaclass changed in Python 3 so choose appropriately.

In [ ]:
class y(object):  # Python 2.x
    __metaclass__ = return_5

In [ ]:
class y(metaclass=return_5):  # Python 3.x
        pass

In [ ]:
y

In [ ]:
type(y)

  We saw how decorators are applied to functions.  They can also be
applied to classes.  What does the following code do?

In [ ]:
def return_6(klass):
    print('Called return_6({!r})'.format(klass))
    return 6

In [ ]:
return_6(None)

In [ ]:
@return_6
class z:
    pass

In [ ]:
z

In [ ]:
type(z)